In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

#wavedata 파형분석

In [ ]:
data_dir = "data/PdM/"

df_mst = pd.read_csv(data_dir + "ics_asset_mst.csv", na_values='\\N')
df_sigdata = pd.read_csv(data_dir + "ics_asset_sigdata.csv", na_values='\\N')
df_status_hist = pd.read_csv(data_dir + "ics_asset_status_hist.csv", na_values='\\N')
df_wavedata = pd.read_csv(data_dir + "ics_asset_wavedata.csv")

df_mst.columns = ['ASSET_ID', 'ASSET_NAME', 'SENSOR_NUMBER']
df_sigdata.columns = ['asset_id', 'created_at',	'temperature', 'voltage',
                      'rms_x', 'rms_y', 'rms_z', 'rms_xyz', 'vel_rms_x', 'vel_rms_y', 'vel_rms_z', 'vel_rms_xyz',
                      'skewness_x', 'skewness_y', 'skewness_z', 'vel_skewness_x', 'vel_skewness_y', 'vel_skewness_z',
                      'kurtosis_x', 'kurtosis_y', 'kurtosis_z', 'vel_kurtosis_x', 'vel_kurtosis_y', 'vel_kurtosis_z',
                      'crest_factor_x', 'crest_factor_y', 'crest_factor_z', 'vel_crest_factor_x', 'vel_crest_factor_y', 'vel_crest_factor_z',
                      'peak_x', 'peak_y', 'peak_z', 'vel_peak_x', 'vel_peak_y', 'vel_peak_z',
                      'peak2peak_x', 'peak2peak_y', 'peak2peak_z', 'vel_peak2peak_x', 'vel_peak2peak_y', 'vel_peak2peak_z']
df_status_hist.columns = ['asset_id', 'time', 'imbalance_health', 'misalignment_health', 'looseness_health', 'bearing_health', 'asset_health', 'CRT_DT']

In [ ]:
df_sigdata.describe()

In [ ]:
# 확인할 그래프 날짜 시간 설정
date_str = "2024-02-06 00:00"
timestamp = pd.Timestamp(date_str)
unix_time = timestamp.timestamp() - 32400

date_str2 = "2024-02-07 00:00"
timestamp2 = pd.Timestamp(date_str2)
unix_time2 = timestamp2.timestamp() - 32400

In [ ]:
# 시간으로 데이터 필터링
filtered_df = df_wavedata[(df_wavedata['created_at'] >= unix_time) & (df_wavedata['created_at'] <= unix_time+86400)]
filtered_df2 = df_wavedata[(df_wavedata['created_at'] >= unix_time2) & (df_wavedata['created_at'] <= unix_time2+86400)]

In [ ]:
# 모터 id로 데이터 필터링
moter_id = "55285839-9b78-48d8-9f4e-573190ace016"
df = filtered_df[filtered_df['asset_id'] == moter_id]
df2 = filtered_df2[filtered_df2['asset_id'] == moter_id]

In [ ]:
# 진폭 데이터 뽑아서 float 리스트로 넣기
def convert_to_float_list(string):
    return [float(x) for x in string.split(',')]
    
df_combined = pd.DataFrame({
    'spectrum_x_amp': df['spectrum_x_amp'].apply(convert_to_float_list),
    'spectrum_y_amp': df['spectrum_y_amp'].apply(convert_to_float_list),
    'spectrum_z_amp': df['spectrum_z_amp'].apply(convert_to_float_list)
})

df_combined2 = pd.DataFrame({
    'spectrum_x_amp': df2['spectrum_x_amp'].apply(convert_to_float_list),
    'spectrum_y_amp': df2['spectrum_y_amp'].apply(convert_to_float_list),
    'spectrum_z_amp': df2['spectrum_z_amp'].apply(convert_to_float_list)
})

In [ ]:
df2

In [ ]:
# 2시간으로 단위로 분리된 하루치 데이터 넣기
df_x = np.empty(0)
df_y = np.empty(0)
df_z = np.empty(0)

for i in range(12):
    df_x = np.append(df_x, df_combined.loc[df_combined.index[0]+i].spectrum_x_amp)
    df_y = np.append(df_y, df_combined.loc[df_combined.index[0]+i].spectrum_y_amp)
    df_z = np.append(df_z, df_combined.loc[df_combined.index[0]+i].spectrum_z_amp)
    
df_new = pd.DataFrame({'spectrum_x_amp': df_x, 'spectrum_y_amp': df_y, 'spectrum_z_amp': df_z})

df_x = np.empty(0)
df_y = np.empty(0)
df_z = np.empty(0)

for i in range(12):
    df_x = np.append(df_x, df_combined2.loc[df_combined2.index[0]+i].spectrum_x_amp)
    df_y = np.append(df_y, df_combined2.loc[df_combined2.index[0]+i].spectrum_y_amp)
    df_z = np.append(df_z, df_combined2.loc[df_combined2.index[0]+i].spectrum_z_amp)
    
df_new2 = pd.DataFrame({'spectrum_x_amp': df_x, 'spectrum_y_amp': df_y, 'spectrum_z_amp': df_z})

print(df_new)
print(df_new2)

In [ ]:
# 인덱스를 3.514배수로
new_index = [unix_time + i*3.514 for i in range(len(df_new))]
new_index = np.round(new_index).astype(int)

new_index2 = [unix_time2 + i*3.514 for i in range(len(df_new))]
new_index2 = np.round(new_index2).astype(int)

df_new.index = new_index
df_new2.index = new_index2
print(df_new)
print(df_new2)

In [ ]:
# 온도 전압 추가

In [ ]:
df_sigdata

In [ ]:
# 시간으로 데이터 필터링
tv_df = df_sigdata[(df_sigdata['created_at'] >= unix_time) & (df_sigdata['created_at'] <= unix_time+86400)]
tv_df2 = df_sigdata[(df_sigdata['created_at'] >= unix_time2) & (df_sigdata['created_at'] <= unix_time2+86400)]

In [ ]:
# 모터 id로 데이터 필터링
tv_df = tv_df[tv_df['asset_id'] == moter_id]
tv_df2 = tv_df2[tv_df2['asset_id'] == moter_id]

In [ ]:
# temperature, voltage, created_at만 추출
tv_df = tv_df[['temperature', 'voltage', 'created_at']].reset_index(drop=True)
tv_df2 = tv_df2[['temperature', 'voltage', 'created_at']].reset_index(drop=True)
print(tv_df)
print(tv_df2)

In [ ]:
# 인덱스 재설정
tv_df = tv_df.set_index('created_at')
tv_df2 = tv_df2.set_index('created_at')

print(tv_df)
print(tv_df2)

In [ ]:
# 인덱스에 맞춰서 columns 추가
df_new.index = df_new.index.astype('int64')
df_new2.index = df_new2.index.astype('int64')

df_new = pd.merge_asof(df_new, tv_df, left_index=True, right_index=True, direction='nearest')
df_new2 = pd.merge_asof(df_new2, tv_df2, left_index=True, right_index=True, direction='nearest')

print(df_new)
print(df_new2)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

ax1 = axes[0]
ax1.plot(df_new['spectrum_x_amp'], label='spectrum_x_amp', alpha=0.8)
ax1.plot(df_new['spectrum_y_amp'], label='spectrum_y_amp', alpha=0.8)
ax1.plot(df_new['spectrum_z_amp'], label='spectrum_z_amp', alpha=0.8)
ax1.set_title('Waveform data visualization 1')
ax1.set_ylim([0, 0.1])
ax1.set_xlabel('Sample index')
ax1.set_ylabel('Amplitude')
ax1.legend(loc='upper left')
ax1.grid(True)

# 온도와 전압을 오른쪽 y축에 추가
ax1_temp = ax1.twinx()
ax1_temp.plot(df_new['temperature'], 'r--', label='Temperature', alpha=0.7)
ax1_temp.set_ylabel('Temperature (°C)', color='r')
ax1_temp.tick_params(axis='y', labelcolor='r')
ax1_temp.set_ylim([4, 50])

ax1_voltage = ax1.twinx()
ax1_voltage.spines['right'].set_position(('outward', 60))  # 오른쪽 y축을 조금 더 떨어뜨리기
ax1_voltage.plot(df_new['voltage'], 'g--', label='Voltage', alpha=0.7)
ax1_voltage.set_ylabel('Voltage (V)', color='g')
ax1_voltage.tick_params(axis='y', labelcolor='g')
ax1_voltage.set_ylim([2.485840, 3.261790])


# 두 번째 서브플롯
ax2 = axes[1]
ax2.plot(df_new2['spectrum_x_amp'], label='spectrum_x_amp', alpha=0.8)
ax2.plot(df_new2['spectrum_y_amp'], label='spectrum_y_amp', alpha=0.8)
ax2.plot(df_new2['spectrum_z_amp'], label='spectrum_z_amp', alpha=0.8)
ax2.set_title('Waveform data visualization 2')
ax2.set_ylim([0, 0.1])
ax2.set_xlabel('Sample index')
ax2.set_ylabel('Amplitude')
ax2.legend(loc='upper left')
ax2.grid(True)

# 온도와 전압을 오른쪽 y축에 추가
ax2_temp = ax2.twinx()
ax2_temp.plot(df_new2['temperature'], 'r--', label='Temperature', alpha=0.7)
ax2_temp.set_ylabel('Temperature (°C)', color='r')
ax2_temp.tick_params(axis='y', labelcolor='r')
ax2_temp.set_ylim([4, 50])

ax2_voltage = ax2.twinx()
ax2_voltage.spines['right'].set_position(('outward', 60))  # 오른쪽 y축을 조금 더 떨어뜨리기
ax2_voltage.plot(df_new2['voltage'], 'g--', label='Voltage', alpha=0.7)
ax2_voltage.set_ylabel('Voltage (V)', color='g')
ax2_voltage.tick_params(axis='y', labelcolor='g')
ax2_voltage.set_ylim([2.485840, 3.261790])

# 레이아웃 조정 및 표시
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].boxplot([df_new['spectrum_x_amp'], df_new['spectrum_y_amp'], df_new['spectrum_z_amp']],
                labels=['spectrum_x_amp', 'spectrum_y_amp', 'spectrum_z_amp'])
axes[0].set_title('Box Plot 1')
axes[0].set_ylabel('amplitude')
axes[0].set_ylim([0, 0.1])
axes[0].grid(True)

axes[1].boxplot([df_new2['spectrum_x_amp'], df_new2['spectrum_y_amp'], df_new2['spectrum_z_amp']],
                labels=['spectrum_x_amp', 'spectrum_y_amp', 'spectrum_z_amp'])
axes[1].set_title('Box Plot 2')
axes[1].set_ylabel('amplitude')
axes[1].set_ylim([0, 0.1])
axes[1].grid(True)

plt.tight_layout()
plt.show()